In [1]:
import pybullet as p
import pybullet_data
import time
import math
import numpy as np
from pinocchio_template import thneed

# Connect to the physics server
physicsClient = p.connect(p.GUI)
p.resetDebugVisualizerCamera(cameraDistance=2.0, cameraYaw=0, cameraPitch=-30, cameraTargetPosition=[0,0,0])

pybullet build time: Jan 29 2025 23:16:28


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.183.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.183.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [2]:
friction_thing_N = 0
gravity = True
position_control = False
disable_velocity_control = not position_control

# Set up the environment
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81*gravity)
planeId = p.loadURDF("/home/a2rlab/Documents/bullet3/data/plane.urdf")

# robotId = p.loadURDF("/home/a2rlab/Documents/bullet3/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf", [0, 0, 0], useFixedBase=1)
robotId = p.loadURDF("/home/a2rlab/Documents/emre/indy-ros2/indy_description/urdf_files/indy7_nolimits.urdf", [0, 0, 0], useFixedBase=1, flags=p.URDF_USE_INERTIA_FROM_FILE)

num_joints = p.getNumJoints(robotId)
print(f"Number of joints: {num_joints}")

# Print joint info
for i in range(num_joints):
    joint_info = p.getJointInfo(robotId, i)
    print(f"Joint {i}, Name: {joint_info[1].decode('utf-8')}, Type: {joint_info[2]}")

if disable_velocity_control:
    p.setJointMotorControlArray(bodyIndex=robotId,
                            jointIndices=[i for i in range(num_joints)],
                            controlMode=p.VELOCITY_CONTROL,
                            forces=friction_thing_N*np.ones(num_joints))

sim_timestep = 0.005
solve_timestep = 0.01
p.setTimeStep(sim_timestep) # Default is 1/240 seconds

t = thneed(dt=solve_timestep)
if not gravity:
    t.gravity_off()

t.max_qp_iters = 5

xs = 0 * np.hstack([np.ones(6), np.zeros(6)])
for i in range(1,7):
    p.resetJointState(robotId, i, xs[i-1], 0)

goal = t.eepos(0.8 * np.ones(6))
start = t.eepos(xs[:t.nq])

goal_trace = []
tracelen = 330
for i in range(tracelen):
    goal_trace.append((1/tracelen)*((tracelen - i)*start + (i)*goal)) # linear interpolation
    # goal_trace.append(goal)
goal_trace = np.array(goal_trace).reshape(-1)

all_xs = []
all_ctrl = []

running_fails = 0
successes = 0
for i in range(300):
    all_xs.append(np.array(xs))
    print(np.linalg.norm(t.eepos(xs[:6]) - goal_trace[3*(i+t.N-1):3*(i+t.N)]))
    

    # if np.linalg.norm(t.eepos(xs[:6]) - goal) < 5e-2:
    #     break

    # t.clean_start()
    # t.shift_start()
    
    t.XU[:t.nx] = xs  # set start state, this helps with finding good min

    if t.sqp(xs, goal_trace[3*i:3*(i+t.N)]):
        print("FAIL")
        running_fails +=1
        if running_fails==t.N-1:
            raise Exception(f"ran out of traj, successes: {successes}")
    else:
        successes +=1
        running_fails = 0
    
    if position_control and 0:
        o = 1
        p.setJointMotorControlArray(bodyIndex=robotId,
                                    jointIndices=[i for i in range(1,7)],
                                    controlMode=p.POSITION_CONTROL,
                                    targetPositions=t.XU[o*t.nxu:o*t.nxu+t.nq], # + 0.3 * np.random.rand(6),
                                    # targetVelocities=t.XU[o*t.nxu+t.nq:o*t.nxu+t.nx],
                                    forces=[1e5 for _ in range(6)]
                                    )
    else:
        o = 0
        trqs = t.XU[o*t.nxu+t.nx:o*t.nxu+t.nxu]
        all_ctrl.append(trqs)
        p.setJointMotorControlArray(bodyIndex=robotId,
                                jointIndices=[i for i in range(1,7)],
                                controlMode=p.TORQUE_CONTROL,
                                forces=trqs)
    
    
    # Step the simulation forward
    for i in range(int(solve_timestep / sim_timestep)):
        p.stepSimulation()
        time.sleep(sim_timestep)
    

    joint_states = p.getJointStates(robotId, [i for i in range(1,7)])
    q = [j[0] for j in joint_states]
    v = [j[1] for j in joint_states]
    torques_applied = [j[3] for j in joint_states] # this is zeroed for torque control


    xs[:t.nq] = q
    xs[t.nq:t.nx] = v


# Disconnect from the physics server
# p.disconnect()

ven = NVIDIA Corporation
ven = NVIDIA Corporation
Number of joints: 8b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: tcp

Joint 0, Name: global, Type: 4
Joint 1, Name: joint0, Type: 0
Joint 2, Name: joint1, Type: 0
Joint 3, Name: joint2, Type: 0
Joint 4, Name: joint3, Type: 0
Joint 5, Name: joint4, Type: 0
Joint 6, Name: joint5, Type: 0
Joint 7, Name: tcp, Type: 4
osqp warm starting: True
0.0853281313029343
0.08805451013864844
0.09075913533454201
0.0934460836292831
0.09610683059957928
0.098747033656113
0.10137037334751167
0.1039675510280839
0.10654394552222908
0.10910263329466399
0.11163518340064747
0.11414950971686537
0.11663752686513731
0.11910582751373747
0.12154132303940485
0.1239549912865366
0.12633499672449927
0.12868978263172332
0.13102048931

ValueError: operands could not be broadcast together with shapes (3,) (0,) 

In [ ]:
all_xs = np.array(all_xs)
all_ctrl = np.array(all_ctrl)

In [ ]:
np.save('xs.npy', all_xs)
np.save('ctrl.npy', all_ctrl)

In [ ]:
friction_thing_N = 0
position_control = True
disable_velocity_control = not position_control

p.resetSimulation()

# Set up the environment
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81*gravity)
planeId = p.loadURDF("/home/a2rlab/Documents/bullet3/data/plane.urdf")

# robotId = p.loadURDF("/home/a2rlab/Documents/bullet3/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf", [0, 0, 0], useFixedBase=1)
robotId = p.loadURDF("/home/a2rlab/Documents/emre/indy-ros2/indy_description/urdf_files/indy7_nolimits.urdf", [0, 0, 0], useFixedBase=1, flags=p.URDF_USE_INERTIA_FROM_FILE)
num_joints = p.getNumJoints(robotId)

if disable_velocity_control:
    p.setJointMotorControlArray(bodyIndex=robotId,
                            jointIndices=[i for i in range(num_joints)],
                            controlMode=p.VELOCITY_CONTROL,
                            forces=friction_thing_N*np.ones(num_joints))
# set joints to 0
for i in range(1,7):
    p.resetJointState(robotId, i, 0, 0)


solve_timestep = 0.01
xss = np.load('xs.npy')
ctrls = np.load('ctrl.npy')
p.setRealTimeSimulation(True)
for i in range(len(xss)):
    if position_control:
        p.setJointMotorControlArray(bodyIndex=robotId,
                                    jointIndices=[i for i in range(1,7)],
                                    controlMode=p.POSITION_CONTROL,
                                    targetPositions=xss[i,:6],
                                    forces=[1e5 for _ in range(6)]
                                    )
    else:
        p.setJointMotorControlArray(bodyIndex=robotId,
                                jointIndices=[i for i in range(1,7)],
                                controlMode=p.TORQUE_CONTROL,
                                forces=ctrls[i,:])
    time.sleep(solve_timestep)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: tcp
